In [ ]:
V4 -> Initial run with AutoGluon public LB score 0.87

In [1]:
import pandas as pd
import numpy as np
import os

In [8]:
!pip install autogluon thinc==8.2.5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of spacy to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to de

In [4]:
data = pd.read_csv('../input/playground-series-s4e10/train.csv')

In [5]:
data.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [9]:
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset('../input/playground-series-s4e10/train.csv')

label = 'loan_status'
predictor = TabularPredictor(label=label,eval_metric='roc_auc').fit(train_data)

Loaded data from: ../input/playground-series-s4e10/train.csv | Columns = 13 / 13 | Rows = 58645 -> 58645
No path specified. Models will be saved in: "AutogluonModels/ag-20241022_061403"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 27 20:43:36 UTC 2024
CPU Count:          4
Memory Avail:       29.97 GB / 31.36 GB (95.6%)
Disk Space Avail:   12.70 GB / 19.52 GB (65.1%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accurac

[1000]	valid_set's binary_logloss: 0.164622


	0.9385	 = Validation score   (roc_auc)
	9.71s	 = Training   runtime
	0.21s	 = Validation runtime
Fitting model: LightGBM ...
	0.9544	 = Validation score   (roc_auc)
	2.42s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9327	 = Validation score   (roc_auc)
	11.6s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9341	 = Validation score   (roc_auc)
	19.8s	 = Training   runtime
	0.5s	 = Validation runtime
Fitting model: CatBoost ...
	0.9518	 = Validation score   (roc_auc)
	109.58s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9222	 = Validation score   (roc_auc)
	5.37s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.919	 = Validation score   (roc_auc)
	5.34s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.9306	 = Validation score   (roc_auc)
	59.13s	 = Training   runtime
	0.05s	 = Val

In [14]:
best_predictor = TabularPredictor.load("AutogluonModels/ag-20241022_061403")

In [15]:

test_data = pd.read_csv('../input/playground-series-s4e10/test.csv')
test_ids = test_data['id']

test_dataset = TabularDataset('../input/playground-series-s4e10/test.csv')
y_pred = best_predictor.predict(test_dataset)

Loaded data from: ../input/playground-series-s4e10/test.csv | Columns = 12 / 12 | Rows = 39098 -> 39098


In [17]:
submission = pd.DataFrame({
    'id': test_ids,
    'loan_status': y_pred
})
submission.to_csv('weightedensemble_submission.csv', index=False)